# TO SUMMARISE:

## We have worked night and day to be ready, streamlit etc
## Now to run any plausibly good model through the same test, and view the results.

## Also, run the models through the same predict set to see their outputs and pick the best 'looker'

In [1]:
#
#
# Doing these postcodes, 
# it is clear the NEW model works better (less Buckingham Palace car park) etc

# ########################      
#    LWB_SMR__POSTCODES_TO_USE
# ########################      

# Suburban London (HH)        SE23 3YL 

# Buckingham Palace           SW1A 1AA

# Edinburgh Castle            EH1 2NG

# Glasgow                     G3 8LZ

# Le Wagon                    E2 8DY

### grabbing models from various places...

In [2]:
# GSTUIL COMMANDS TO DOWNLOAS VARIOUS MODELS INTO THE CWD:

# model names include the date the run was started on (might run overnight...) 

# gsutil cp gs://lwb-solar-my-roof/models/2206var_hh_vertex_models.zip 2206var_hh_vertex_models.zip
# gsutil cp gs://lwb-solar-my-roof/models/aa_vertex_220616_day8_model_unfreeze_from_params.zip aa_vertex_220616_day8_model_unfreeze_from_params.zip 
# gsutil cp gs://lwb-solar-my-roof/models/220615_josh_checkpoint_VGG16_Combo_Dice_BCE_unfrozen_block.h5 220615_josh_checkpoint_VGG16_Combo_Dice_BCE_unfrozen_block.h5

In [3]:
model_names_dict ={
    'm1_old': '220612_Josh_model_vertexAI_07_FULL_dataset_dice.h5',
    'm2_new': '220615_josh_checkpoint_VGG16_Combo_Dice_BCE_unfrozen_block.h5'
}

In [4]:
model_names_dict['m1_old'],model_names_dict['m2_new'] 

('220612_Josh_model_vertexAI_07_FULL_dataset_dice.h5',
 '220615_josh_checkpoint_VGG16_Combo_Dice_BCE_unfrozen_block.h5')

In [5]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import tensorflow as tf
# from sklearn.model_selection import train_test_split
from lwb_smr.data import GetData, LoadDataSets # for: create_tensor_slicer, process_data_inc_autotune
from lwb_smr.params import BATCH_SIZE, EPOCHS, LOSS
from lwb_smr.params import EXPERIMENT_NAME, EXPERIMENT_TAGS
from lwb_smr.params import TEST_RUN, DATA_AUGMENTATION

LOAD_MODEL_FROM = 'local' # or set to: 'gcp', or 'new_arch'

MLFLOW_URI = "https://mlflow.lewagon.ai/"
UNET_INPUT_SHAPE = (224,224,3)

# for these values, see params.py
EXPERIMENT_NAME = EXPERIMENT_NAME
EXPERIMENT_TAGS = EXPERIMENT_TAGS

DATA_AUGMENTATION = DATA_AUGMENTATION
LOSS = LOSS

BATCH_SIZE = BATCH_SIZE 
EPOCHS = EPOCHS


2022-07-01 15:27:48.347700: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-01 15:27:48.347725: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
# run this cell to confirm the raw_data path:

!ls '../../raw_data/'

where_the_data_went.txt


In [7]:
root_path = '../../raw_data/'

In [8]:
###########################
# GetDat input parameters #
###########################

train_path = root_path + "train_RGB_tiles_jpeg/"
mask_path = root_path + "train_mask_tiles_jpeg/"
# we don't need a test_path, do we?? mask_path = root_path + "train_mask_tiles_jpeg/"

input_image_size = (250,250)
train_percent = 0.7
validation_percent = 0.25
test_percent = 0.05

In [9]:
get_data = GetData(train_path,mask_path,train_percent,validation_percent,test_percent)


print(f'TEST_RUN in params.py is set to: {TEST_RUN}')
print(f'BATCH_SIZE = {BATCH_SIZE} and EPOCHS = {EPOCHS}')

TEST_RUN in params.py is set to: True
BATCH_SIZE = 8 and EPOCHS = 2


In [10]:
# custom loss functions:

# [note, also defined in SMR_Model() below]

def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

def loss_combo_dice_bce(y_true, y_pred):
    # JACK
    def dice_loss(y_true, y_pred):
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)

        return 1 - numerator/denominator

    y_true = tf.cast(y_true, tf.float32)
    o = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred) + dice_loss(y_true, y_pred)

    return tf.reduce_mean(o)

custom_objects_dict = {
                'LOSS': 'dice_loss',
}

custom_objects_dict_OLD = {
                'dice_loss': 'dice_loss',
}

custom_objects_dict_NEW = {
                'loss_combo_dice_bce': 'loss_combo_dice_bce'
}

# LOAD OF MODEL from GCP if you need:
## set the path... 

In [11]:
# NOT THIS ONE!!!!   
#               LOADING_MODEL_GSUTIL = 'gs://lwb-solar-my-roof/models/220610_full_model_vgg16_10e_20e_more.h5'
# THIS ONE >>>>>>> 
#               i.e. with the dice loss
if LOAD_MODEL_FROM == 'gcp':
    LOADING_MODEL_GSUTIL = 'gs://lwb-solar-my-roof/models/220611_VGG16_Dice_20e_in_shape_224x224x3.h5'
    loaded_model = tf.keras.models.load_model(LOADING_MODEL_GSUTIL, custom_objects=custom_objects_dict)


# LOCAL LOAD OF MODEL:
## set the path... 

In [12]:
model_names_dict['m1_old'], model_names_dict['m2_new']

('220612_Josh_model_vertexAI_07_FULL_dataset_dice.h5',
 '220615_josh_checkpoint_VGG16_Combo_Dice_BCE_unfrozen_block.h5')

In [13]:
# WORKS WELL FOR LOCAL LOAD:
# model_load_string = 'input_your_local_path_to_model_here'
model_local_path = '/home/hsth/code/hughharford/lwb_smr/raw_data/models/'
modelOLD_load_string = model_local_path + model_names_dict['m1_old']
modelNEW_load_string = model_local_path + model_names_dict['m2_new']

if LOAD_MODEL_FROM == 'local':
    loadedOLD_model = tf.keras.models.load_model(modelOLD_load_string, custom_objects=custom_objects_dict_OLD)
    loadedNEW_model = tf.keras.models.load_model(modelNEW_load_string, custom_objects=custom_objects_dict_NEW)

OSError: No file or directory found at /home/hsth/code/hughharford/lwb_smr/raw_data/models/220612_Josh_model_vertexAI_07_FULL_dataset_dice.h5

In [14]:
# # Check its architecture
# loaded_model.summary()

# OR CREATE A NEW MODEL WITH UNFROZEN LAYERS

In [21]:
if LOAD_MODEL_FROM == 'new_arch':
    from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
    from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.utils import plot_model
    import tensorflow as tf

    from tensorflow.keras.applications import VGG16

    class SMR_Model_unfrozen():
        ''' creating vgg16 model that last block is trainable
        '''

        def __init__(self, input_shape, model_path_to_file=''):
            self.input_shape = input_shape
            if model_path_to_file:
                self.model_to_load = model_path_to_file

        def convolution_block(self, inputs, num_filters):
            ''' simple UNET convolution block with BatchNormalisation '''

            # convolution layer 1 of the block
            x = Conv2D(num_filters, (3,3), padding='same')(inputs)  # padding='same' to avoid cut-down with conv
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            # convolution layer 2 of the block
            x = Conv2D(num_filters, (3,3), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            # max pooling not used here as just the bridge

            return x

        def decoder_block(self, inputs, skip_tensor, num_filters):
            ''' decoder block for UNET '''
            # adds in the skips with concatenate
            x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs) # stride important here to up-sample
            x = Concatenate()([x, skip_tensor])     # bringing in skip layer
            x = self.convolution_block(x, num_filters)

            return x

        def build_vgg16_unet(self):
            ''' build vgg-16 '''

            inputs = Input(self.input_shape)

            # see actual VGG-16 here: https://github.com/keras-team/keras/blob/v2.9.0/keras/applications/vgg16.py#L43-L227
            vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
            # vgg16.summary()
            # Unfreeze the last block of VGG16
            for layer in vgg16.layers[:15]:
                layer.trainable = False

            # check that layers have been set accordingly:
            for i, layer in enumerate(vgg16.layers):
                print(i, layer.name, layer.trainable)


            ''' Encoder - skip layers '''
            skip1 = vgg16.get_layer('block1_conv2').output #  256 x 256, 64 filters in vgg16
            skip2 = vgg16.get_layer('block2_conv2').output #  128 x 128, 128 filters in vgg16
            skip3 = vgg16.get_layer('block3_conv3').output #   64 x 64, 256 filters in vgg16
            skip4 = vgg16.get_layer('block4_conv3').output #   32 x 32, 512 filters in vgg16
            # display('skip4: ' + str(skip4.shape))

            # only need to specify the skip layers, as VGG16 is an Encoder
            # Therefore, VGG16 comes built with MaxPool2d, so we don't specify

            ''' Bridge '''
            bridge = vgg16.get_layer('block5_conv3').output # 16 x 16, with 512 filters in vgg16
            # display('bridge: ' + str(bridge.shape))


            ''' Decoder '''
            d1 = self.decoder_block(bridge, skip4, 512) #  512 filters, as per the bridge
            d2 = self.decoder_block(d1, skip3, 256) #  256 filters
            d3 = self.decoder_block(d2, skip2, 128) #  128 filters
            d4 = self.decoder_block(d3, skip1, 64)  #   64 filters

            ''' Output '''
            outputs = Conv2D(1, (1,1), padding='same', activation='sigmoid')(d4)

            model = Model(inputs, outputs, name='VGG16_UNET_LastBlockUnfrozen_ComboLossDICE_BCE')

            return model


        def loss_combo_dice_bce(self, y_true, y_pred):
            # JACK
            def dice_loss(y_true, y_pred):
                y_pred = tf.math.sigmoid(y_pred)
                numerator = 2 * tf.reduce_sum(y_true * y_pred)
                denominator = tf.reduce_sum(y_true + y_pred)

                return 1 - numerator/denominator

            y_true = tf.cast(y_true, tf.float32)
            o = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred) + dice_loss(y_true, y_pred)

            return tf.reduce_mean(o)

        def compile_model(self, m):
            ''' with accuracy, binaryIoU, AuC '''
            # metrics
            threshold = 0.5
            binaryIoU = tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=threshold)
            AuC = tf.keras.metrics.AUC()

            # loss
            #self.dice_loss = ...

            # Compile Model
            m.compile(
                        loss=self.loss_combo_dice_bce,
                        optimizer='adam',
                        metrics=['accuracy', binaryIoU, AuC]
                        )
            return m

In [22]:

# get_vgg16 = SMR_Model_unfrozen(UNET_INPUT_SHAPE)
# model_vgg16 = get_vgg16.build_vgg16_unet()
# model_vgg16_compiled = get_vgg16.compile_model(model_vgg16)

In [23]:
# history = model_vgg16_compiled.summary()

# TENSOR SLICE DATA LOADER

In [24]:
## NOW TRYING: create_tensor_slicer
# 

# x_path = '../../../raw_data/train_RGB_tiles_jpeg/'
# x_images = os.listdir(x_path)
# y_path = '../../../raw_data/train_mask_tiles_jpeg/'
# y_masks = os.listdir(y_path)

# root_path = '../../raw_data/'
# folders = ['train_RGB_tiles_jpeg', 'train_mask_tiles_jpeg']
# folder_path = [f'{root_path}{folder}' for folder in folders]
# folder_path

# train_images, train_mask = [], []
# train_images =[f'../../../raw_data/train_RGB_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[0])]
# # for i, filename in enumerate(os.listdir(folder_path[0])):
# #     if i == 2*BATCH_SIZE: break
# #     train_images.append(f'../../../raw_data/train_RGB_tiles_jpeg/{filename}')
    
# train_mask = [f'../../../raw_data/train_mask_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[1])]
# # for i, filename in enumerate(os.listdir(folder_path[1])):
# #     if i == 2*BATCH_SIZE: break
# #     train_mask.append(f'../../../raw_data/train_mask_tiles_jpeg/{filename}')

In [25]:
## NOW TRYING: create_tensor_slicer
# 
# # train_images.sort()
# train_mask.sort()

# train_df = pd.DataFrame()
# # train_df['file_path'] = train_images
# train_df['image_path'] = train_images
# train_df['mask_path'] = train_mask

# len(train_df), train_df.head()

In [26]:
# train_df.info()

In [27]:
data_dict = LoadDataSets(
    root_path + "image_datasets_csv/train_dataset.csv",
    root_path + "image_datasets_csv/validation_dataset.csv",
    root_path + "image_datasets_csv/test_dataset.csv").load_datasets()
data_dict.keys()

Datasets match, proceed


dict_keys(['train_x', 'train_y', 'val_x', 'val_y', 'test_x', 'test_y'])

In [28]:
ds_train, ds_val, ds_test = get_data.create_tensor_slicer(data_dict)


In [29]:
ds_test

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [30]:
# process_data_inc_autotune(self, ds_train, ds_val=None, ds_test=None,
#                                   data_augmentation=1):
ds_train, ds_val, ds_test = get_data.process_data_inc_autotune(
    ds_train, ds_val, ds_test, data_augmentation=DATA_AUGMENTATION)


In [31]:
ds_test

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None))>

# model.py

In [32]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications import VGG16

class SMR_Model():
    ''' creating our first lwb_smr models '''

    def __init__(self, input_shape):
        self.input_shape = input_shape

    def get_latest_model(self):
        model = self.build_vgg16_unet(self.input_shape)
        model = self.compile_model(model)

        return model

    def convolution_block(self, inputs, num_filters):
        ''' simple UNET convolution block with BatchNormalisation '''

        # convolution layer 1 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(inputs)  # padding='same' to avoid cut-down with conv
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # convolution layer 2 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # max pooling not used here as just the bridge

        return x

    def decoder_block(self, inputs, skip_tensor, num_filters):
        ''' decoder block for UNET '''
        # adds in the skips with concatenate
        x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs) # stride important here to up-sample
        x = Concatenate()([x, skip_tensor])     # bringing in skip layer
        x = self.convolution_block(x, num_filters)

        return x

    def build_vgg16_unet(self, input_shape):
        ''' build vgg-16 '''

        inputs = Input(input_shape)

        # see actual VGG-16 here: https://github.com/keras-team/keras/blob/v2.9.0/keras/applications/vgg16.py#L43-L227
        vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
        # vgg16.summary()
        vgg16.trainable = False

        ''' Encoder - skip layers '''
        skip1 = vgg16.get_layer('block1_conv2').output #  256 x 256, 64 filters in vgg16
        skip2 = vgg16.get_layer('block2_conv2').output #  128 x 128, 128 filters in vgg16
        skip3 = vgg16.get_layer('block3_conv3').output #   64 x 64, 256 filters in vgg16
        skip4 = vgg16.get_layer('block4_conv3').output #   32 x 32, 512 filters in vgg16
        # display('skip4: ' + str(skip4.shape))

        # only need to specify the skip layers, as VGG16 is an Encoder
        # Therefore, VGG16 comes built with MaxPool2d, so we don't specify

        ''' Bridge '''
        bridge = vgg16.get_layer('block5_conv3').output # 16 x 16, with 512 filters in vgg16
        # display('bridge: ' + str(bridge.shape))


        ''' Decoder '''
        d1 = self.decoder_block(bridge, skip4, 512) #  512 filters, as per the bridge
        d2 = self.decoder_block(d1, skip3, 256) #  256 filters
        d3 = self.decoder_block(d2, skip2, 128) #  128 filters
        d4 = self.decoder_block(d3, skip1, 64)  #   64 filters

        ''' Output '''
        outputs = Conv2D(1, (1,1), padding='same', activation='sigmoid')(d4)

        model = Model(inputs, outputs, name='first_VGG16_UNET')

        return model
    
    def dice_loss(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.math.sigmoid(y_pred)
        numerator = 2 * tf.reduce_sum(y_true * y_pred)
        denominator = tf.reduce_sum(y_true + y_pred)

        return 1 - numerator / denominator

    def loss_combo_dice_bce(self, y_true, y_pred):
        # JACK
        def dice_loss(y_true, y_pred):
            y_pred = tf.math.sigmoid(y_pred)
            numerator = 2 * tf.reduce_sum(y_true * y_pred)
            denominator = tf.reduce_sum(y_true + y_pred)

            return 1 - numerator/denominator

        y_true = tf.cast(y_true, tf.float32)
        o = tf.nn.sigmoid_cross_entropy_with_logits(y_true, y_pred) + dice_loss(y_true, y_pred)

        return tf.reduce_mean(o)

    def compile_model(self, m):
        ''' with accuracy, binaryIoU, AuC '''
        # metrics
        threshold = 0.5
        binaryIoU = tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=threshold)
        AuC = tf.keras.metrics.AUC()

        # loss
        #self.dice_loss = ...
        
        # Compile Model
        m.compile(
                    loss=self.dice_loss,
                    optimizer='adam',
                    metrics=['accuracy', binaryIoU, AuC]
                    )
        return m


# utils.py

In [33]:
# mlflow
#
# and others...

import mlflow
from mlflow.tracking import MlflowClient
from memoized_property import memoized_property

MLFLOW_URI = "https://mlflow.lewagon.ai/"

class PushMLFlow():
    '''
        MLFLOW_URI = "https://mlflow.lewagon.ai/"
        EXPERIMENT_NAME = "[UK] [LONDON] [SOLAR_ROOF] TEST RUN" # template
        EXPERIMENT_TAGS = {
            'USER': 'TEST_user',
            'RUN NAME': 'TEST_evaluation of models',
            'VERSION': '1.0.1',
            'LOSS': 'dice'
            'DESCRIPTION': 'testing MLFlow Pipeline. Model - basic U-Net structure, 2 epochs, 15 images'
        }
    '''

    def __init__(self, experiment_name, experiment_tags):
        self.experiment_name = experiment_name
        self.experiment_tag = experiment_tags

    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id, tags=self.experiment_tag)

    def mlflow_log_param(self, key, value):
        self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)


In [34]:
# trainer.py

In [35]:

import os
import tensorflow as tf

# from lwb_smr.CustomDataLoader import CustomDataLoader
# from lwb_smr.model import SMR_Model
# from lwb_smr.utils import PushMLFlow

from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import AUC, IoU

        
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

class Trainer():
    def __init__(self):
        pass

    def set_model(self, loss=dice_loss):
        self.loss = loss
        # Instantiate Model
        # our_input_shape = (224,224,3)

        #### getVGG16 = SMR_Model(UNET_INPUT_SHAPE)
        #### self.model = getVGG16.get_latest_model()
        
        # TO RUN A NEWLY CREATED UNFROZEN LAYERS VGG16, USE
        # self.model = model_vgg16_compiled
        
        # TO LOAD FROM ABOVE, USE:
        ####                      self.model = loaded_model # see additions above
        # self.model = loadedOLD_model
        self.model = loadedNEW_model
        # see compile in SMR_Model

    def start_mlflow(self):
        p = PushMLFlow(EXPERIMENT_NAME, EXPERIMENT_TAGS)
        return p # returns a class instance of PushMLFlow
    
    def run(self):

        print(80*'-')
        print('------SETTING FOR DATA RUN------')

        ### 
        ##
        #
        # customdata = self.just_get_the_data_loaded()

        print(80*'-')
        print('------MODEL RUNNING------')

        # set mflow      
        self.MFLOW = self.start_mlflow() # class instance of MLFLOW

        
        # set model
        self.set_model()

        # mc = ModelCheckpoint('220611_checkpoint_VGG16_Dice.h5', save_best_only=True) # could put path here
        # es = EarlyStopping(patience=15, restore_best_weights=True)
        # self.history = self.model.fit(
        #     ds_train,
        #     validation_data=ds_val,
        #     batch_size=BATCH_SIZE,
        #     epochs=EPOCHS,
        #     callbacks=[mc, es]
        #     )

        model_path_and_filename = 'day10evaluate_OLD_model.h5'
        self.model.save(model_path_and_filename)

        self.loss_n_metrics = ['old model: loss', 'accuracy', 'binaryIoU', 'AuC']
        # self.MFLOW.mlflow_log_param('loss', self.loss)
        self.MFLOW.mlflow_log_param('accuracy', 'accuracy')
        self.MFLOW.mlflow_log_param('binaryIoU', 'binaryIoU')
        self.MFLOW.mlflow_log_param('AuC', 'AuC')


        print(80*'=')
        print('------MODEL RUN SUCCESFULLY COMPLETED------')

        self.evaluate()

    def evaluate(self):
        print(80*'-')
        print('------MODEL EVALUATING------')
        results = self.model.evaluate(ds_test)
        for result in results:
            self.MFLOW.mlflow_log_metric('metric X val output', result)
        print(80*'=')
        print('------MODEL EVALUATED------')

if __name__ == '__main__':
    pass
    t = Trainer()
    t.run()

--------------------------------------------------------------------------------
------SETTING FOR DATA RUN------
--------------------------------------------------------------------------------
------MODEL RUNNING------
------MODEL RUN SUCCESFULLY COMPLETED------
--------------------------------------------------------------------------------
------MODEL EVALUATING------


TypeError: in user code:

    File "/home/hsth/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "/home/hsth/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/hsth/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "/home/hsth/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 1501, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/hsth/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "/home/hsth/.pyenv

In [ ]:
########
####                        DICE             DICE             DICE             DICE             DICE             DICE             DICE             DICE
###
#
#        NOTES: with DICE LOSS driving the model now (this is a sample run. 
#                                                                          Again, clear that binaryIoU metric is showing increases, as with all other metrics.
#                                              (val) DICE LOSS reducing
#                                              (val) ACCURACY increasing
#                                              (val) binaryIoU increasing
#                                              (val) AUC increasing
#
##
###
####
########

In [ ]:
t.history

In [ ]:
t.__dict__